# Topic Modeling with Gensim

A **topic model** is an abstraction of the major topics contained in a corpus of texts. "Topic" in this context simply means a pattern of co-occurring words. The assumption is that if there are clearly identified patterns of co-occurring words, those patterns of co-occurring words reveal a latent structure in the corpus of texts. In short, a topic model is a representation of the major themes or structures of a corpus of texts.

`Gensim` is a popular Python library for building topic models. In this notebook we will use `Gensim` to build a topic model of Gibbon's _Decline and Fall of the Roman Empire_. After building a topic model, we will then use `pyLDAvis` to visualize the model so we can evaluate its usefulness.

I highly recommend that you read through `Gensim`'s [documentation](https://radimrehurek.com/gensim/auto_examples/index.html#core-tutorials-new-users-start-here). Much of the code below is adapted from that source.

## Set up

**NOTE**: one of the Python libraries we are using (`pyLDAvis`) can cause problems. Be sure to do the installations in the order that you see them below.

In [1]:
! pip install funcy

In [2]:
! pip install tzdata

In [3]:
! pip install --no-dependencies pyLDAvis

                                              0.0/2.6 MB ? eta -:--:--
                                              0.0/2.6 MB ? eta -:--:--
                                              0.0/2.6 MB ? eta -:--:--
                                              0.0/2.6 MB 326.8 kB/s eta 0:00:08
     --                                       0.2/2.6 MB 1.2 MB/s eta 0:00:03
     ------                                   0.4/2.6 MB 2.4 MB/s eta 0:00:01
     -----------                              0.7/2.6 MB 3.2 MB/s eta 0:00:01
     ----------------                         1.1/2.6 MB 4.2 MB/s eta 0:00:01
     ----------------------                   1.5/2.6 MB 5.2 MB/s eta 0:00:01
     ---------------------------              1.8/2.6 MB 5.4 MB/s eta 0:00:01
     -------------------------------          2.0/2.6 MB 5.6 MB/s eta 0:00:01
     ---------------------------------------  2.6/2.6 MB 6.6 MB/s eta 0:00:01
     ---------------------------------------- 2.6/2.6 MB 6.4 MB/s eta 0:00:00


In [4]:
! pip install wget

In [7]:
! pip install gensim

                                              0.0/24.0 MB ? eta -:--:--
                                              0.0/24.0 MB ? eta -:--:--
                                              0.0/24.0 MB ? eta -:--:--
                                             0.0/24.0 MB 326.8 kB/s eta 0:01:14
                                              0.2/24.0 MB 1.3 MB/s eta 0:00:19
                                              0.6/24.0 MB 3.3 MB/s eta 0:00:08
     -                                        1.0/24.0 MB 4.4 MB/s eta 0:00:06
     --                                       1.5/24.0 MB 5.9 MB/s eta 0:00:04
     ---                                      2.2/24.0 MB 7.6 MB/s eta 0:00:03
     ----                                     2.9/24.0 MB 8.8 MB/s eta 0:00:03
     ------                                   3.7/24.0 MB 10.2 MB/s eta 0:00:02
     -------                                  4.6/24.0 MB 11.9 MB/s eta 0:00:02
     ---------                                5.6/24.0 MB 12.8 MB/s et

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.4.1 requires numexpr, which is not installed.


In [30]:
from collections import defaultdict
import wget
from gensim import corpora, models
import pandas as pd
import pyLDAvis.gensim
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import requests
import json
import math

## Upload data

### Class example

In [8]:
# url = 'https://raw.githubusercontent.com/msaxton/nlp-data/main/gibbon_sections.csv'
# file_name = wget.download(url)
# df = pd.read_csv(file_name)
# df.head()

### Upload your own data sets

If you are using Google Colab:

In [ ]:
# uploaded = files.upload()

In [ ]:
# file_name = # <-- for example: 'data.csv'
# df = pd.read_csv(io.BytesIO(uploaded[file_name]))

If you are using Jupyter Notebooks:

In [26]:
url = 'https://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1770&date2=1963&proxtext="China"+"Chinese"&x=0&y=0&dateFilterType=yearRange&rows=20&searchType=basic&format=json'
response = requests.get(url)
raw = response.text
results = json.loads(raw)
results.keys()

dict_keys(['totalItems', 'endIndex', 'startIndex', 'itemsPerPage', 'items'])

In [28]:
print(results['items'][0])

{'sequence': 62, 'county': [None], 'edition': None, 'frequency': 'Daily', 'id': '/lccn/sn83045462/1944-08-13/ed-1/seq-62/', 'subject': ['Washington (D.C.)--fast--(OCoLC)fst01204505', 'Washington (D.C.)--Newspapers.'], 'city': ['Washington'], 'date': '19440813', 'title': 'Evening star. [volume]', 'end_year': 1972, 'note': ['"From April 25 through May 24, 1861 one sheet issues were published intermittently owing to scarcity of paper." Cf. Library of Congress, Photoduplication Service.', 'Also issued on microfilm from Microfilming Corp. of America and the Library of Congress, Photoduplication Service.', 'Archived issues are available in digital format as part of the Library of Congress Chronicling America online collection.', 'Publisher varies: Noyes, Baker & Co., <1867>; Evening Star Newspaper Co., <1868->', "Suspended Jan. 1-6, 1971 because of a machinists' strike."], 'state': ['District of Columbia'], 'section_label': '', 'type': 'page', 'place_of_publication': 'Washington, D.C.', 'sta

In [31]:
total_pages = math.ceil(results['totalItems'] / results['itemsPerPage'])
print(total_pages)

3158


In [32]:
data = []

In [33]:
start_date = '1770'
end_date = '1865'
search_term = "China"+"Chinese"
state = ''

In [34]:
for i in range(1, 10):  
    url = (f'https://chroniclingamerica.loc.gov/search/pages/results/?state={state}&date1={start_date}'
           f'&date2={end_date}&proxtext={search_term}&x=16&y=8&dateFilterType=yearRange&rows=20'
           f'&searchType=basic&format=json&page={i}')  # f-string
    response = requests.get(url)
    raw = response.text
    print(f'page {i} status code:', response.status_code)  # checking for errors
    results = json.loads(raw)
    items_ = results['items']
    for item_ in items_:
        row_data = {}
        try:
          row_data['title'] = item_['title_normal']
        except:
          row_data['city'] = "none"
        try:
          row_data['city'] = item_['city']
        except:
          row_data['city'] = "none"
        try:
          row_data['date'] = item_['date']
        except:
          row_data['date'] = "none"
        try:
          row_data['raw_text'] = item_['ocr_eng']
        except:
          row_data['raw_text'] = 'none'
    data.append(row_data)

page 1 status code: 200
page 2 status code: 200
page 3 status code: 200
page 4 status code: 200
page 5 status code: 200
page 6 status code: 200
page 7 status code: 200
page 8 status code: 200
page 9 status code: 200


In [35]:
df = pd.DataFrame.from_dict(data)

In [40]:
df.head()

,title,city,date,raw_text
0,davenport gazette.,[Davenport],18530707,"mr""®\nSANDERS k MYIS,\nFBOPBIETOBS' BUSINESS C..."
1,yazoo democrat.,[Yazoo City],18530105,THE YAZOO DEMOCRAT\n1'\nf\nPublished Weekly\nV...
2,washington sentinel.,[Washington],18540405,fetal aitb personal.\nThe Hippodrome.?The prep...
3,lancaster gazette.,[Lancaster],18480324,n i\n' S. I:\nill\nn\n' ' 1\n;!:!)\n111\n! i\n...
4,yazoo democrat.,[Yazoo City],18520825,TRY AGAIN.\nTV CORBETT takes leave to return h...


## Prepare data for topic model
The Python library we are going to use to make our topic model requires the data to be in a form of a list. Within that list, each "document" is also a list. So it looks something like this:

`[
  ['This is document 1'],
  ['This is document 2'],
  ['This is document 3']
]`

In [41]:
# extract the data out of the DataFrame
documents = df['raw_text'].to_list()

In [43]:
len(documents[0])

45959

`Gensim` needs each document to be tokenized. We can use [list comprehension](https://www.w3schools.com/python/python_lists_comprehension.asp) to quickly achieve this result. When complete, our data will now look like this:

`[
  ['This', 'is', 'document', '1'],
  ['This', 'is', 'document', '2'],
  ['This', 'is', 'document', '3'],
]`

In [44]:
# tokenize - the syntax below will create a list of lists
texts =[
    [word for word in document.lower().split()]
    for document in documents
]

It takes a lot of preparation to build a useful topic model. An important part of that preparation is to eliminate "noise" from you model. One way to do this is to remove pieces of data that are irrelevant. Here we will remove tokens that only occur once. **You may want to adjust this as you refine your topic model.**

In [45]:
# create a count of each token
frequency = defaultdict(int)
for text in texts:
  for token in text:
    frequency[token] += 1

In [46]:
# remove words that appear only 1 time
texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

## Build topic model

`Gensim` is built around [four core concepts](https://radimrehurek.com/gensim/auto_examples/core/run_core_concepts.html#core-concepts):
- **document**: an individual text. In our case, this is an individual section from Gibbon.
- **corpus**: a collection of documents. In our case, this is all the sections from Gibbon put together.
- **vector**: a mathematically convenient representation of a document. Basically, each word in the document is given a numerical id. This allows `Gensim` to do faster calculations behind the scene.
- **model**: an algorithm for transforming vectors from one representation to another. In our case, this will be the LDA model we build.


### Basic topic model



In [47]:
# create a dictionary based off our texts
# The dictionary maps each token to a unique integer id
dictionary = corpora.Dictionary(texts)

In [48]:
# create a corpus based off our dictionary and our texts
corpus = [dictionary.doc2bow(text) for text in texts]

In [49]:
# build LDA model
lda_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20, passes=50)

In [50]:
# explore topics
lda_model.print_topics()

[(0,
  '0.000*"the" + 0.000*"of" + 0.000*"and" + 0.000*"to" + 0.000*"in" + 0.000*"for" + 0.000*"a" + 0.000*"be" + 0.000*"at" + 0.000*"that"'),
 (1,
  '0.000*"of" + 0.000*"the" + 0.000*"and" + 0.000*"in" + 0.000*"to" + 0.000*"a" + 0.000*"for" + 0.000*"be" + 0.000*"on" + 0.000*"at"'),
 (2,
  '0.000*"the" + 0.000*"of" + 0.000*"and" + 0.000*"to" + 0.000*"in" + 0.000*"a" + 0.000*"be" + 0.000*"at" + 0.000*"for" + 0.000*"by"'),
 (3,
  '0.000*"the" + 0.000*"of" + 0.000*"and" + 0.000*"to" + 0.000*"in" + 0.000*"a" + 0.000*"for" + 0.000*"at" + 0.000*"that" + 0.000*"is"'),
 (4,
  '0.076*"the" + 0.051*"of" + 0.034*"and" + 0.027*"to" + 0.023*"in" + 0.014*"a" + 0.012*"be" + 0.011*"at" + 0.010*"on" + 0.010*"."'),
 (5,
  '0.083*"the" + 0.036*"of" + 0.032*"and" + 0.028*"a" + 0.027*"to" + 0.019*"in" + 0.014*"was" + 0.010*"his" + 0.010*"with" + 0.010*"that"'),
 (6,
  '0.000*"the" + 0.000*"of" + 0.000*"and" + 0.000*"to" + 0.000*"in" + 0.000*"a" + 0.000*"for" + 0.000*"be" + 0.000*"that" + 0.000*"at"'),
 (7,

In [51]:
# Find topics in each document
lda_model.get_document_topics(corpus[0])

[(5, 0.99984163)]

In [59]:
# visualize
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, mds='mmds')
vis

C:\Users\vivia\miniconda3\Lib\site-packages\sklearn\manifold\_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.155867 -0.118505       1        1  64.431933
14    -0.298982 -0.087320       2        1  12.343236
5     -0.195867 -0.228038       3        1  12.216603
4     -0.095622 -0.291993       4        1  10.993840
0      0.046643  0.045363       5        1   0.000899
11     0.046648  0.045367       6        1   0.000899
18     0.046647  0.045367       7        1   0.000899
17     0.046644  0.045364       8        1   0.000899
16     0.046647  0.045367       9        1   0.000899
15     0.046647  0.045367      10        1   0.000899
12     0.046648  0.045367      11        1   0.000899
10     0.046647  0.045366      12        1   0.000899
1      0.046648  0.045367      13        1   0.000899
9      0.046647  0.045367      14        1   0.000899
8      0.046647  0.045367      15        1   0.000899
7      0.046645  0.045365      16        1   0.000899
6      0.046647  0.045366      17        1   0.000899
3      0.046642  0.045362      18        1   0.000899
2      0.046647  0.045367      19        1   0.000899
19     0.046645  0.045365      20        1   0.000899, topic_info=       Term        Freq        Total Category  logprob  loglift
1405    the  3884.00000  3884.000000  Default  30.0000  30.0000
1022     of  2308.00000  2308.000000  Default  29.0000  29.0000
111     and  1742.00000  1742.000000  Default  28.0000  28.0000
1458     to  1420.00000  1420.000000  Default  27.0000  27.0000
53        a   931.00000   931.000000  Default  26.0000  26.0000
...     ...         ...          ...      ...      ...      ...
1551     we     0.00009   210.997070  Topic20  -8.5211  -3.0518
133     are     0.00009   216.186367  Topic20  -8.5211  -3.0761
1571  which     0.00009   223.221420  Topic20  -8.5211  -3.1082
999     not     0.00009   199.046285  Topic20  -8.5211  -2.9935
176    been     0.00009   170.005207  Topic20  -8.5211  -2.8358

[1323 rows x 6 columns], token_table=      Topic      Freq Term
term                      
13        1  0.818882    &
13        2  0.012042    &
13        3  0.024085    &
13        4  0.144509    &
1642      1  0.459444    '
...     ...       ...  ...
4803      1  0.993808    |
4806      1  0.991564    —
4808      1  0.990145    ’
4809      1  0.994703    “
4813      1  0.993804    ■

[529 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 15, 6, 5, 1, 12, 19, 18, 17, 16, 13, 11, 2, 10, 9, 8, 7, 4, 3, 20])

### Tf-idf topic model

In [53]:
# initialize a tfidf model
tfidf = models.TfidfModel(corpus)

In [54]:
# make a new corpus based on the tfidf model
corpus_tfidf = tfidf[corpus]

In [55]:
# here we build our topic model
lda_model_tfidf = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=20, passes=50)
corpus_lda = lda_model_tfidf[corpus_tfidf]

In [56]:
lda_model_tfidf.print_topics()

[(0,
  '0.000*"ship," + 0.000*"sheriff" + 0.000*"skin" + 0.000*"sixty" + 0.000*"simple" + 0.000*"signed" + 0.000*"sible" + 0.000*"shore" + 0.000*"society" + 0.000*"sheriffs"'),
 (1,
  '0.001*"“" + 0.001*"■" + 0.001*"’" + 0.001*"|" + 0.001*"fairfax" + 0.001*"confederate" + 0.001*"‘" + 0.001*"buren," + 0.001*"capt." + 0.001*"van"'),
 (2,
  '0.000*"ship," + 0.000*"sheriff" + 0.000*"skin" + 0.000*"sixty" + 0.000*"simple" + 0.000*"signed" + 0.000*"sible" + 0.000*"shore" + 0.000*"society" + 0.000*"sheriffs"'),
 (3,
  '0.000*"ship," + 0.000*"sheriff" + 0.000*"skin" + 0.000*"sixty" + 0.000*"simple" + 0.000*"signed" + 0.000*"sible" + 0.000*"shore" + 0.000*"society" + 0.000*"sheriffs"'),
 (4,
  '0.002*"yazoo" + 0.001*"1852." + 0.001*"probate" + 0.001*"township" + 0.001*"quarter" + 0.001*"east-half" + 0.001*"1852," + 0.001*"south-west" + 0.001*"north-east" + 0.001*"quarter,"'),
 (5,
  '0.000*"ship," + 0.000*"sheriff" + 0.000*"skin" + 0.000*"sixty" + 0.000*"simple" + 0.000*"signed" + 0.000*"sible"

In [57]:
# Find topics in each document
lda_model_tfidf.get_document_topics(corpus_tfidf[0])

[(0, 0.031738747),
 (1, 0.031738747),
 (2, 0.031738747),
 (3, 0.031738747),
 (4, 0.031738747),
 (5, 0.031738747),
 (6, 0.031738747),
 (7, 0.031738747),
 (8, 0.03173875),
 (9, 0.031738747),
 (10, 0.3969638),
 (11, 0.031738747),
 (12, 0.031738747),
 (13, 0.031738747),
 (14, 0.031738747),
 (15, 0.031738747),
 (16, 0.031738747),
 (17, 0.03173875),
 (18, 0.031738747),
 (19, 0.031738747)]

In [60]:
# visualize
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_tfidf, corpus_tfidf, dictionary, mds='mmds')
vis

C:\Users\vivia\miniconda3\Lib\site-packages\sklearn\manifold\_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.000603  0.001188       1        1  15.033291
17     0.000699 -0.001447       2        1   9.515009
11     0.000707  0.001864       3        1   9.491660
9     -0.001033 -0.001115       4        1   9.101027
13    -0.001058  0.000399       5        1   8.958785
1      0.000550  0.000896       6        1   8.776630
8      0.001624 -0.000498       7        1   8.749648
4     -0.002034 -0.000269       8        1   8.711087
18     0.000097 -0.000084       9        1   1.805239
16     0.000095 -0.000085      10        1   1.805239
15     0.000090 -0.000088      11        1   1.805239
14     0.000096 -0.000085      12        1   1.805239
0      0.000091 -0.000084      13        1   1.805239
12     0.000097 -0.000085      14        1   1.805239
7      0.000097 -0.000085      15        1   1.805239
6      0.000100 -0.000082      16        1   1.805239
5      0.000101 -0.000081      17        1   1.805239
3      0.000096 -0.000086      18        1   1.805239
2      0.000093 -0.000087      19        1   1.805239
19     0.000093 -0.000087      20        1   1.805239, topic_info=          Term      Freq     Total Category  logprob  loglift
2517     yazoo  0.000000  0.000000  Default  30.0000  30.0000
3753   mexican  0.000000  0.000000  Default  29.0000  29.0000
1699         ?  0.000000  0.000000  Default  28.0000  28.0000
3742  marshall  0.000000  0.000000  Default  27.0000  27.0000
4919        ??  0.000000  0.000000  Default  26.0000  26.0000
...        ...       ...       ...      ...      ...      ...
25          .1  0.000813  0.044716  Topic20  -8.5216   0.0072
26           0  0.000813  0.044716  Topic20  -8.5216   0.0072
27          0.  0.000813  0.044716  Topic20  -8.5216   0.0072
28          00  0.000813  0.064049  Topic20  -8.5216  -0.3522
29          01  0.000813  0.044716  Topic20  -8.5216   0.0072

[1100 rows x 6 columns], token_table=Empty DataFrame
Columns: [Topic, Freq, Term]
Index: [], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 18, 12, 10, 14, 2, 9, 5, 19, 17, 16, 15, 1, 13, 8, 7, 6, 4, 3, 20])